In [1]:
import numpy as np
from lib.defaults import Main
import json
# from lib.controller import dynamic, decompose
from lib.lqr import classical_lqr
from lib.simple import Sim
from scipy.linalg import solve_lyapunov, solve

In [2]:
# params I need 
main = Main()
a = main.a
r2 = main.r2_dynamic
pmd_slice = main.pmd_slice
taus = main.taus
tau = main.tau
# prms
n = main.n
n_e = main.n_e

with open('setup_prms.json') as f:
    prms = json.load(f)
c = np.array(prms['c'])

In [3]:
tau_z, tau_y = main.taus
q = Sim().q_mov(c)

# I pick out codes from controller.dynamic

In [4]:
n, _ = a.shape
n_pmd = len(pmd_slice)
n_i = n - n_e
tau_z, tau_y = taus
const_y = (tau / tau_y) * np.eye(n_e)
const_z = (tau / tau_z) * np.eye(n_e)

yx = 0.1
x = np.random.binomial(1, yx, size=(n_e, n_e)) / (yx * n_e)
yx = np.hstack((x, np.zeros((n_e, n_i))))

a = np.block([
    [a, np.zeros((n, n_e)), np.zeros((n, n_e))],
    [(tau / tau_y) * yx, -const_y, np.zeros((n_e, n_e))],
    [np.zeros((n_e, n)), const_z, -const_z]
])

b = np.block([
    [np.eye(n)],
    [np.zeros((n_e + n_e, n))]
])

c = np.hstack((np.zeros((n_e, n)), np.zeros((n_e, n_e)), np.eye(n_e)))

q = np.block([
    [q, np.zeros((n, n_e)), np.zeros((n, n_e))],
    [np.zeros((n_e, n)), np.zeros((n_e, n_e)), np.zeros((n_e, n_e))],
    [np.zeros((n_e, n)), np.zeros((n_e, n_e)), np.zeros((n_e, n_e))]
])

In [5]:
class Adam:
    def __init__(self, learning_rate=5, beta1=0.9, beta2=0.999, epsilon=1e-8, weight_decay=0, lr_schedule=None):
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.weight_decay = weight_decay
        self.m = None
        self.v = None
        self.t = 0
        self.lr_schedule = lr_schedule
    
    def update(self, params, gradients):
        if self.m is None:
            self.m = [np.zeros_like(p) for p in params]
            self.v = [np.zeros_like(p) for p in params]
        
        self.t += 1
        
        # Compute the first and second moment estimates
        for i in range(len(params)):
            self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * gradients[i]
            self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * gradients[i]**2
        
        # Bias correction for first and second moment estimates
        m_hat = [self.m[i] / (1 - self.beta1**self.t) for i in range(len(params))]
        v_hat = [self.v[i] / (1 - self.beta2**self.t) for i in range(len(params))]
        
        # Learning rate schedule
        if self.lr_schedule is not None:
            lr = self.learning_rate * self.lr_schedule(self.t)
        else:
            lr = self.learning_rate
        
        # Weight decay
        gradients = [gradients[i] + self.weight_decay * params[i] for i in range(len(params))]
        
        # Update parameters
        params = [params[i] - lr * m_hat[i] / (np.sqrt(v_hat[i]) + self.epsilon) for i in range(len(params))]
        
        return params


# I pick out codes from lqr.output_lqr

In [6]:
n, _ = a.shape
iden = np.eye(n)
clo, raw = (n_pmd, n_e)
npara = clo * raw
bt = b.T
ct = c.T

def k_of(z):
    return z @ c

def s_of(acl):
    return solve_lyapunov(acl, -iden)

def p_of(acl, k):
    return solve_lyapunov(acl.T, -(q + (r2 * k.T @ bt @ b @ k)))

def grad_z(k, p, s):
    return 2 * bt @ (p + (r2 * b @ k)) @ s @ ct   


def f_df(z):
    z = np.reshape(z, (clo, raw))
    k = k_of(z)
    acl = a + (b @ k) 
    p = p_of(acl, k)  
    return np.trace(p)

def grad_func(z):
    z = np.reshape(z, (clo, raw))
    k = k_of(z)
    acl = a + (b @ k) 
    s = s_of(acl)
    p = p_of(acl, k)
    g_ = grad_z(k, p, s)    
    return g_.reshape(npara)


z0 = classical_lqr(r2, q, a, b) @ solve(c @ c.T, c).T
z0 = z0.reshape(npara)

n_iter = 23
for i in range(n_iter):                        
    l = f_df(z0)
    z0 = np.array(Adam().update([z0], grad_func(z0))[0])
    print(f'iteration {i+1}/{n_iter} | train loss: {l:.6f}')
#     if l < 1e-6:
#         print(f'train loss: {l:.6f}')
#         break

iteration 1/23 | train loss: 503.293667
iteration 2/23 | train loss: 524.149179
iteration 3/23 | train loss: 499.975336
iteration 4/23 | train loss: 475.156968
iteration 5/23 | train loss: 450.179861
iteration 6/23 | train loss: 425.139578
iteration 7/23 | train loss: 400.067788
iteration 8/23 | train loss: 374.978023
iteration 9/23 | train loss: 349.877036
iteration 10/23 | train loss: 324.768573
iteration 11/23 | train loss: 299.654880
iteration 12/23 | train loss: 274.537385
iteration 13/23 | train loss: 249.417040
iteration 14/23 | train loss: 224.294503
iteration 15/23 | train loss: 199.170244
iteration 16/23 | train loss: 174.044609
iteration 17/23 | train loss: 148.917854
iteration 18/23 | train loss: 123.790179
iteration 19/23 | train loss: 98.661736
iteration 20/23 | train loss: 73.532647
iteration 21/23 | train loss: 48.403009
iteration 22/23 | train loss: 23.272900
iteration 23/23 | train loss: -1.857615


# The loss function is computed negative

In [7]:
print(f'iteration {i+1}/{n_iter} | train loss: {l:.6f}')

iteration 23/23 | train loss: -1.857615


In [8]:
z0 = np.reshape(z0, (clo, raw))
k = k_of(z0)
acl = a + (b @ k) 
print(p_of(acl, k)) 

[[ 2.16292319  1.27170805 -0.65291126 ...  0.86361112  0.86361112
   0.86361112]
 [ 1.27170805  2.2762881   0.50678537 ...  0.59251077  0.59251077
   0.59251077]
 [-0.65291126  0.50678537  1.80502186 ...  0.77379659  0.77379659
   0.77379659]
 ...
 [ 0.86361112  0.59251077  0.77379659 ... -2.90612802 -2.90612802
  -2.90612802]
 [ 0.86361112  0.59251077  0.77379659 ... -2.90612802 -2.90612802
  -2.90612802]
 [ 0.86361112  0.59251077  0.77379659 ... -2.90612802 -2.90612802
  -2.90612802]]


# adam_optimizer is correct

In [9]:
from scipy.optimize import rosen, rosen_der

In [10]:
class Adam:
    def __init__(self, learning_rate=1e-1, beta1=0.9, beta2=0.999, epsilon=1e-8, weight_decay=0, lr_schedule=None):
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.weight_decay = weight_decay
        self.m = None
        self.v = None
        self.t = 0
        self.lr_schedule = lr_schedule
    
    def update(self, params, gradients):
        if self.m is None:
            self.m = [np.zeros_like(p) for p in params]
            self.v = [np.zeros_like(p) for p in params]
        
        self.t += 1
        
        # Compute the first and second moment estimates
        for i in range(len(params)):
            self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * gradients[i]
            self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * gradients[i]**2
        
        # Bias correction for first and second moment estimates
        m_hat = [self.m[i] / (1 - self.beta1**self.t) for i in range(len(params))]
        v_hat = [self.v[i] / (1 - self.beta2**self.t) for i in range(len(params))]
        
        # Learning rate schedule
        if self.lr_schedule is not None:
            lr = self.learning_rate * self.lr_schedule(self.t)
        else:
            lr = self.learning_rate
        
        # Weight decay
        gradients = [gradients[i] + self.weight_decay * params[i] for i in range(len(params))]
        
        # Update parameters
        params = [params[i] - lr * m_hat[i] / (np.sqrt(v_hat[i]) + self.epsilon) for i in range(len(params))]
        
        return params


In [11]:
z0 = np.zeros(200)
n_iter = 100
for i in range(n_iter):                        
    l = rosen(z0)
    z0 = np.array(Adam().update([z0], rosen_der(z0))[0])
    print(f'iteration {i+1}/{n_iter} | train loss: {l:.6f}')
    if l < 1e-6:
        print(f'train loss: {l:.6f}')
        break

iteration 1/100 | train loss: 199.000000
iteration 2/100 | train loss: 322.379999
iteration 3/100 | train loss: 636.799998
iteration 4/100 | train loss: 975.099998
iteration 5/100 | train loss: 1217.879999
iteration 6/100 | train loss: 1293.500000
iteration 7/100 | train loss: 1178.080002
iteration 8/100 | train loss: 895.500003
iteration 9/100 | train loss: 517.400003
iteration 10/100 | train loss: 163.180003
iteration 11/100 | train loss: 0.000000
train loss: 0.000000
